# Topic Modeling

In machine learning and natural language processing, a topic model is a type of **statistical model for discovering the abstract "topics" that occur in a collection of documents.**

Topic modeling is a frequently used text-mining tool for discovery of **hidden semantic structures** in a text body. 

**How do we capture the hidden semantic structures?**

**Intuitively**, given that a document is about a particular topic, one would expect particular words to appear in the document more or less frequently: 
- "dog" and "bone" will appear more often in documents about dogs, 
- "cat" and "meow" will appear in documents about cats, and 
- "the" and "is" will appear equally in both. 

A document typically concerns multiple topics in different proportions; thus, in a document that is 10% about cats and 90% about dogs, there would probably be about 9 times more dog words than cat words. 

**The "topics" produced by topic modeling techniques are clusters of similar words.**

**A topic model** captures this intuition in a mathematical framework, which allows examining a set of documents and discovering, based on the statistics of the words in each, what the topics might be and what each document's balance of topics is.

Source: [Topic Modeling](https://en.wikipedia.org/wiki/Topic_model)

## Applications in industry

- Publications/Newspapers have several articles (documents) across various topics. These documents could be from either sports, medicine, technology, health, business, media, entertainment. They would want to categorize each document into a specific topic. We could also place multiple topics to same document, for instance, an article speaking about Pele could have topics such as sports, football, and celebrity.

- Banks or any enterprise would have multiple documents and they would want to automatically categorize the documents as information, customer grievance, enquiry, sales, product description etc.

- Web based companies such as Stack Overflow, Quora would want to automate the tagging/categorization of the question. 

- This is used as pre-step to supervised techniques. For instance, for chat applications, an interaction is categorized by topic modeling and is assigned to the customer agent, however, if this is wrongly assigned, the category is updated and over time we get more information of the interaction-category (right). Once sufficient data is available, it becomes a multi-label multi-classification problem.

## Dataset (Unsupervised Technique)



We shall use the 20 newsgroups dataset. 

[Dataset Description](http://qwone.com/~jason/20Newsgroups/)

It comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The split between the train and test set is based upon a messages posted before and after a specific date.

However, since we are doing topic modeling, we shall not use the topic labels but will use for verification/evaluation.

In [0]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

from pprint import pprint
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


**For this exercise, we shall take only 4 newsgroups.**

In [0]:
import numpy as np

from sklearn import decomposition
from scipy import linalg

import matplotlib.pyplot as plt

In [0]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
remove = ('headers', 'footers', 'quotes')

newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

### Data Exploration

In [0]:
newsgroups_train.filenames.shape, newsgroups_train.target.shape

((2034,), (2034,))

In [0]:
print("\n--------------------------------".join(newsgroups_train.data[:3]))

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
--------------------------------

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
for centuries.
-----------------------------

In [0]:
np.array(newsgroups_train.target_names)[newsgroups_train.target[:3]]

array(['comp.graphics', 'talk.religion.misc', 'sci.space'], dtype='<U18')

## Topic Modeling Implementation

### Intuition of using Matrix Decomposition/Matrix Factorization

Factorization is to get the factors. E.g. 36 -> 2 * 2 * 3 * 3. So, the prime factors are 2 and 3. 

Similarly, we try to find the factors of the matrix. For instance, in the below image, background and the people are two factors, which could be separated as shown.

![Matrix_Decomposition](https://i.imgur.com/n1xxjlx.png)


### Singular Value Decomposition

Before we get into the theory let us understand the output, so that we can relate back to the concept.

[topic_modeling_output](../excel/topic_modeling_literature.xlsx)


The singular value decomposition of an __m * n__ matrix **M** is a factorization of the form $U \Sigma V^T$ , 

where,
- __U__ is an _m * m_ matrix, 

- $\Sigma$ is an _m * n_ rectangular diagonal matrix with non-negative real numbers on the diagonal and 

- **V** is _n * n_ matrix.

    
The diagonal entries of $\Sigma$ are known as the **singular values of M**. 

The **columns of U and the columns of V** are called the **left-singular vectors and right-singular vectors of M**.

**Singular Vectors** -> Those vectors/matrices whose determinant is 0 i.e. it does not have an inverse matrix and hence singular (remember it as single).

![SVD](https://i.imgur.com/M1fqdVe.png)

Source: [SVD_Wikipedia](https://en.wikipedia.org/wiki/Singular_value_decomposition)





As per the above image,

$V^*$ is same as $V^T$.

$U$ . $U^T$ i.e. dot product of U with itself is one.

$V$ . $V^T$ i.e. dot product of U with itself is one.

**Better visualization of the matrix factors:**

![SVD_Partial_matrix](https://i.imgur.com/jZCx0Kx.png)
(source: [Facebook Research: Fast Randomized SVD](https://research.fb.com/fast-randomized-svd/))

**Properties of U, s and V matrices?**

The SVD algorithm factorizes a matrix into one matrix (U matrix) with **orthonormal columns** and one with **orthonormal rows** ($V^T$ matrix), along with a __diagonal matrix (s matrix)__, which contains the **relative importance** of each factor.

The columns of U matrix and rows of the $V^T$ matrix are **orthonormal** to each other.

**What is orthonormal?**

In linear algebra, two vectors are orthonormal if they are orthogonal and unit vectors.
- Orthogonal -> The vectors are perpendicular to each other i.e. the dot product is 0.
- Unit Vectors -> The length/magnitude of the vectors is unity i.e. dot product of the vector with itself is 1.

[Orthonormal - wiki](https://en.wikipedia.org/wiki/Orthonormality#Simple_example)

**But what does orthogonal mean in the context of topics?**

We would clearly expect that the words that appear most frequently in one topic would appear less frequently in the other - otherwise that word wouldn't make a good choice to separate out the two topics. Therefore, we expect the topics to be **orthogonal** i.e. less over-lapping.

**What does s matrix represent?**

**s matrix** has non-negative values in descending order and it represents the importance of the topics.

#### Implementation - SVD

Notice that this representation does not take into account word order or sentence structure.  It's an example of a **bag of words** approach. So let us get the CountVectorizer or TfidfVectorizer to get the term-document matrices.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
vectorizer = CountVectorizer(stop_words='english')

In [0]:
vectors = vectorizer.fit_transform(newsgroups_train.data).todense() # (documents, vocab)
vectors.shape

(2034, 26576)

In [0]:
vocab = np.array(vectorizer.get_feature_names())
print(vocab.size)

26576


In [0]:
vocab[5000:5020]

array(['brow', 'brown', 'browning', 'browns', 'browse', 'browsing',
       'bruce', 'bruces', 'bruise', 'bruised', 'bruises', 'brunner',
       'brush', 'brushmapping', 'brushmaps', 'brussel', 'brutal',
       'brutally', 'brute', 'bryan'], dtype='<U80')

In [0]:
%time U, s, Vt = linalg.svd(vectors, full_matrices=False)

Wall time: 11.3 s


**NOTE**

**full_matrices = True -> would get matrix U.shape as m * m i.e. 2034 * 2034 and Vt.shape as 26576 * 26576.**

**When full_matrices = False, we use k=min(m,n). The dimensions would be m * k (2034 * 2034) and k * n (2034 * 26576).**

See **Appendix** (below) for further explanation.

In [0]:
print(U.shape, s.shape, Vt.shape)

(2034, 2034) (2034,) (2034, 26576)


In [0]:
num_top_words=8

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [0]:
show_topics(Vt[:10])  # We would have 2034 topics and if we want to reduce it to just few topics, we would need to use truncated SVD or NMF.

['ditto critus propagandist surname galacticentric kindergarten surreal imaginative',
 'jpeg gif file color quality image jfif format',
 'graphics edu pub mail 128 3d ray ftp',
 'jesus god matthew people atheists atheism does graphics',
 'image data processing analysis software available tools display',
 'god atheists atheism religious believe religion argument true',
 'space nasa lunar mars probe moon missions probes',
 'image probe surface lunar mars probes moon orbit',
 'argument fallacy conclusion example true ad argumentum premises',
 'space larson image theory universe physical nasa material']

We get topics that match the kinds of clusters we would expect! This is despite the fact that this is an unsupervised algorithm - which is to say, we never actually told the algorithm how our documents are grouped.

### Non-negative Matrix Factorization

Just like we did for SVD, let us see how the output of NMF looks like.

[NMF](../excel/topic_modeling_literature.xlsx)

We see that instead of 3 matrices we just have 2 matrices and also we see that there is **NO** negative values. That was the idea behind NMF. 

In the SVD sheet of the excel, we see lots of negative values, which were not interpretable. With NMF, we just have positive values that enables interpretation, one of the reasons for its popularity.

Rather than constraining our factors to be *orthogonal*, another idea would to constrain them to be *non-negative*. NMF is a factorization of a non-negative data set $V$: $$ V = W H$$ into non-negative matrices $W,\; H$.


![NMF](https://i.imgur.com/nwiECNi.png)


The product of W and H **approximates** the non-negative matrix V and hence it is a non-exact factorization that factors into one skinny positive matrix and one short positive matrix.  

In [0]:
m,n = vectors.shape
d = 5  # num topics

In [0]:
clf = decomposition.NMF(n_components=d, random_state=1)

W1 = clf.fit_transform(vectors)
H1 = clf.components_

In [0]:
show_topics(H1)

['jpeg image gif file color images format quality',
 'edu graphics pub mail 128 ray ftp send',
 'space launch satellite nasa commercial satellites year market',
 'jesus god people matthew atheists does atheism said',
 'image data available software processing ftp edu analysis']

In [0]:
vectorizer_tfidf = TfidfVectorizer(stop_words='english')
vectors_tfidf = vectorizer_tfidf.fit_transform(newsgroups_train.data) # (documents, vocab)

In [0]:
W1 = clf.fit_transform(vectors_tfidf)
H1 = clf.components_

In [0]:
show_topics(H1)

['people don think just like objective say morality',
 'graphics thanks files image file program windows know',
 'space nasa launch shuttle orbit moon lunar earth',
 'ico bobbe tek beauchaine bronx manhattan sank queens',
 'god jesus bible believe christian atheism does belief']

**Notes:**
- For NMF, matrix needs to be at least as tall as it is wide, or we get an error with fit_transform
- Can use df_min in CountVectorizer to only look at words that were in at least k of the split texts

### Truncated SVD (Latent Semantic Analysis)

**Problem with SVD**

SVD gave back 2034 topics -- so how do we get few topics, like the one we saw in the excel sheet? We need to use Truncated SVD.

**Remedy:**

Truncated SVD implements a variant of singular value decomposition (SVD) that only computes the **largest singular values, where is a user-specified parameter.**

When truncated SVD is applied to term-document matrices (as returned by CountVectorizer or TfidfVectorizer), this transformation is known as latent semantic analysis (LSA), because it transforms such matrices to a **semantic space of low dimensionality.**

**Relationship of Truncated SVD and PCA:**

TruncatedSVD is very similar to PCA, but differs in that it works on sample matrices  directly instead of their covariance matrices. When the columnwise (per-feature) means of  are subtracted from the feature values, truncated SVD on the resulting matrix is equivalent to PCA.

In Scikit-learn package, Contrary to PCA, this estimator does not center the data before computing the singular value decomposition. This means it can work with scipy.sparse matrices efficiently.

In [0]:
from sklearn.utils.extmath import randomized_svd

In [0]:
%time u, s, v = randomized_svd(vectors, 4)

Wall time: 4.15 s


In [0]:
show_topics(v[:4])

['jpeg image edu file graphics images gif data',
 'jpeg gif file color quality image jfif format',
 'space jesus launch god people satellite matthew atheists',
 'jesus god matthew people atheists atheism does graphics']

## Appendix

### Full and Reduced SVD (Reduced SVD is not same as Truncated)

Remember how we were calling `np.linalg.svd(vectors, full_matrices=False)`?  We set `full_matrices=False` to calculate the reduced SVD.  For the full SVD, both U and V are **square** matrices, where the extra columns in U form an orthonormal basis i.e. columns are orthonormal (but zero out when multiplied by extra rows of zeros in S).


We know that S matrix is rectangular in shape ( m * n ), but the output shows square matrix (m * m), that is because we do not consider them as those are rows of 0 and only the diagonal elements upto n (considering m >= n) are filled. Since these are zeros, multiplying part of U with zeros will yield to zeros and hence that part of the U matrix is cut-off. Hence, we see reduced SVD.

![Full_Reduced_SVD](https://i.imgur.com/74GzBOZ.png)

![Reduced SVD](https://i.imgur.com/Ya4rLMH.png)